In [1]:
# addprocs(30);
addprocs(7);

In [2]:
require(:POMDPModels)
require(:POMCP)

In [3]:
@everywhere begin
    using POMDPModels
    using POMCP
    using POMDPs
    using POMDPToolbox
    import POMDPToolbox.PreviousObservation
    import POMCP.init_V
    import POMDPs: action, updater
end

In [4]:
# N = 100000;
N = 100;
problem = BabyPOMDP(-5, -10);

In [5]:
function est_reward(problem, policy, belief, N; eps=0.01)
    sum = @parallel (+) for i in 1:N
        sim_rng = MersenneTwister(i)
        sim = POMDPToolbox.RolloutSimulator(rng=sim_rng, initial_state=false, eps=eps)
        up = updater(policy)
        POMDPs.simulate(sim, problem, policy, up, convert_belief(up,belief))
    end
    return sum/N;
end

est_reward (generic function with 1 method)

In [6]:
# Feed when Crying (Expected Reward for this nearly-optimal policy is -17.14)
# Test from earlier this week with 5000 experiments: -16.72
@time est_reward(problem, FeedWhenCrying(), PreviousObservation(false), N)

-15.061634270536405

1.246955 seconds (451.05 k allocations: 19.464 MB, 1.75% gc time)


This is better than in the crying babies test because epsilon is large and, more importantly, it gets a notcrying observation on the first step every time

In [7]:
# Random
pol_rng = MersenneTwister(7)
@time est_reward(problem, RandomPolicy(problem, rng=pol_rng), BoolDistribution(0.5), N)

-31.610260048602548

0.764749 seconds (450.54 k allocations: 18.844 MB, 0.55% gc time)


In [8]:
# POMCP with FWC rollout policy
# test from earlier this week with 5000 experiments: -16.77
rng = MersenneTwister(2)

solver = POMCPSolver(rollout_solver=FeedWhenCrying(),
                    eps=0.01,
                    c=10.0,
                    tree_queries=300, 
                    rng=rng,
                    updater=updater(problem))
                    

policy = solve(solver, problem)

@time est_reward(problem, policy, BoolDistribution(0.0), N)

 12

-14.515122407995722

.721739 seconds (178.51 k allocations: 7.895 MB)


In [9]:
# POMCP with Random rollout policy
rng = MersenneTwister(2)
rollout_pol_rng = MersenneTwister(2)

solver = POMCPSolver(rollout_solver=RandomPolicy(problem, rng=rollout_pol_rng),
                     eps=0.01,
                     c=10.0,
                     tree_queries=300, 
                     rng=rng,
                     updater=updater(problem))

policy = solve(solver, problem)

@time est_reward(problem, policy, BoolDistribution(0.0), N)

 12

-15.581507973916004

.330935 seconds (14.91 k allocations: 1.222 MB)


In [10]:
# Optimal policy for these particular problem parameters:
# if the belief that the baby is hungry is over .28206, then feed (see DMU book)
@everywhere begin
    type OptBabyPolicy <: POMDPs.Policy end
    function action(p::OptBabyPolicy, b::BoolDistribution, a=false)
        a = b.p>0.28206
        return a
    end
    updater(::OptBabyPolicy) = updater(BabyPOMDP(-5,-10))
end
@time est_reward(problem, OptBabyPolicy(), BoolDistribution(0.0), N)

-14.396481928274111

0.149372 seconds (46.25 k allocations: 2.332 MB)
